### ✅ Anatomía de la Request y la Response

Para analizar la Request y la Response de una página web, podemos usar la herramienta de desarrollo de Chrome.

1. Visita una página web culquiera dentro de un `navegador web`. Para el ejemplo, vamos a usar las páginas web de [https://www.up.edu.mx/](https://www.up.edu.mx/), o [https://wrsbyte.com](https://wrsbyte.com).
2. Abre la herramienta de desarrollo de Chrome. Para ello, presiona `F12` o `Ctrl+Shift+I` (Windows) o `Cmd+Option+I` (Mac). O puedes hacer clic derecho sobre la página y seleccionar "Inspeccionar".
3. En la herramienta de desarrollo, selecciona la pestaña "Network".

Si quieres realizar peticiones manuales controladas, puedes usar curl o wget.

1. Abre la terminal.
2. Ejecuta el siguiente comando:

```bash
# Para obtener la Response de la Request
  curl -I https://www.up.edu.mx

# Para obtener el HTML de la Request
curl https://www.up.edu.mx

# Para indicar cabeceras: User Agent y Content-Type
curl -H "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36" -H "Content-Type: text/html; charset=utf-8" https://www.up.edu.mx
```

3. Analiza la Response.

También puedes hacerlo con software especializado como Postman.

1. Descarga e instala Postman. Puedes hacerlo desde [https://www.postman.com/downloads/](https://www.postman.com/downloads/).
2. Abre Postman.
3. En la pestaña "Request", escribe la URL de la página web que quieres analizar.

Algunos métodos HTTP son GET, POST, PUT, DELETE, PATCH, etc. Pueden tener un body o no. 
Cuando tienen body, la petición curl luce así:

```bash
curl 'https://wrsbyte.com/_actions/sendEmail/' \
  -H 'accept: application/json' \
  -H 'accept-language: en-US,en;q=0.9,es-US;q=0.8,es;q=0.7' \
  -H 'content-type: application/json' \
  -H 'dnt: 1' \
  -H 'origin: https://wrsbyte.com' \
  -H 'priority: u=1, i' \
  -H 'referer: https://wrsbyte.com/en/' \
  -H 'sec-ch-ua: "Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  -H 'sec-fetch-dest: empty' \
  -H 'sec-fetch-mode: cors' \
  -H 'sec-fetch-site: same-origin' \
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36' \
  --data-raw '{"name":"wilmer","email":"ejemplo@gmail.com","message":"hola desde la up"}'
```



### ✨ Métodos HTTP Esenciales

Los métodos indican la acción que el cliente desea realizar sobre el recurso. En scraping, nos enfocamos en los siguientes:

| Método      | Propósito                                                      | Uso en Scraping                                                                                         |
|-------------|------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|
| GET         | Solicitar un recurso específico. Idempotente (seguro de repetir). | El 90% del scraping: obtener páginas HTML, JSON de APIs, imágenes.                                       |
| POST        | Enviar datos al servidor para crear o actualizar un recurso. No idempotente. | Simular el envío de formularios (ej. login, búsqueda con filtros complejos) donde los parámetros viajan en el Body. |
| HEAD        | Solicitar solo los headers de un recurso, sin el Body.          | Validación eficiente: verificar si un recurso existe o si ha cambiado (Content-Length, Last-Modified) sin descargar la página completa. |
| PUT/DELETE  | Rara vez usados en scraping, se usan para modificar o eliminar recursos. | Solo si se interactúa con APIs propias o de terceros, no para scraping de sitios web públicos.           |


### 🛠️ Códigos de Estado y Manejo de Errores

El Código de Estado en la respuesta es crítico para determinar si el scraping fue exitoso y cómo proceder si no lo fue.

| Rango | Código            | Significado                                 | Manejo en Scraping                                                                                         |
|-------|--------------------|---------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| 2xx   | 200 OK             | La solicitud fue exitosa.                   | Éxito. Proceder con el parsing del Body.                                                                    |
| 3xx   | 301/302            | Redirección permanente/temporal.            | El scraper debe seguir el nuevo Location Header (requests lo hace por defecto, pero es útil entenderlo).   |
| 4xx   | 403 Forbidden      | El servidor deniega el acceso.              | Bloqueo. Suele deberse a User-Agent incorrecto, IP baneada o cookies inválidas. Requiere rotación de IP/headers. |
| 4xx   | 404 Not Found      | El recurso no existe en el servidor.        | Error de URL. Registrar el fallo y continuar.                                                               |
| 5xx   | 500 Internal Server Error | Error del servidor al procesar la solicitud. | Retry: implementar reintentos con espera (time.sleep()) antes de descartar.                                 |


### 🪂 Headers Esenciales: Simular un Navegador

Para evadir defensas básicas (firewalls o filtros de bots), nuestro scraper debe enviar Headers que hagan parecer que la solicitud proviene de un navegador legítimo.

 * **User-Agent**: Es el más importante. Identifica el tipo y la versión del cliente. Un User-Agent genérico de Python (python-requests/2.28.1) es una bandera roja.

     **Ejemplo**: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36

 * **Referer**: Indica la URL desde donde el usuario "llegó" a la página actual. Útil para simular un flujo de navegación real.

 * **Accept-Encoding**: Indica los formatos de compresión que el cliente puede manejar (ej. gzip, deflate).

 * **Content-Type**: Indica el tipo de contenido que el cliente puede manejar (ej. text/html, application/json).

 * **Cookie**: Contiene los datos de sesión. Es vital para mantener el estado de login, seguir rutas de compra o evitar ser bloqueado por sistemas de control de sesión.